# A4 - Common Analysis  

During the last two years we all have been experiencing a global pandemic. This has been tragic and disruptive to many countries and has taken a deep personal toll on many individuals and their families.  
One aspect that has been hard to miss in the last two years is the datafication of the pandemic. That is, many aspects of the individual toll of the pandemic have been collected, aggregated and re-represented as data. This datafication gives us the privilege to examine the pandemic from potentially many different perspectives to understand how it has changed lives and how it has changed society. To be honest, we are actually at the very beginning of understanding and comprehending these impacts.  
During our Course Project we are going to begin taking a look at some of the social aspects of the pandemic by conducting a human centered data science analysis of some available COVID-19 data. In this Assignment A4: Common Analysis, every student in the course will work from the same datasets. Students will be assigned to analyze data for one specific County of the United States.

## Step 0 - Data Acquisition
The common analysis research question will require several different datasets. You will need:  
- The RAW_us_confirmed_cases.csv file from the [Kaggle repository of John Hopkins University COVID-19 data.](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university?select=RAW_us_confirmed_cases.csv)  
- The [CDC dataset](https://data.cdc.gov/Policy-Surveillance/U-S-State-and-Territorial-Public-Mask-Mandates-Fro/62d6-pm5i) of masking mandates by county.  
- The New York Times mask compliance [survey data.](https://github.com/nytimes/covid-19-data/tree/master/mask-use)  

The majority of this data is by US County by Day. The mask compliance is a single shot estimator that gives you a compliance estimate for every County in the US. You should carefully review the data descriptions that accompany these datasets. They each have some interesting caveats. As well, some of them are explicit with regard to the way you should interpret missing data.  

My assignment:  

| County | State | Area_km^2| Area_mi^2 | Population_2020_Census | Population_Increase_from_2010 | County Seat |  
| ------ | ----- | -------- | --------- | ---------------------- | ----------------------------- | ----------- |  
| Orange | California | 2,047.56 | 790.57 | 3,186,989 | 176,757 | Santa Ana |  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [2]:
# Set up
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Show all columns
pd.set_option('display.max_columns', None)

# Set filepaths
RAW_DATA_PATH = '../data_raw/'
CLEAN_DATA_PATH = '../data_clean/'

In [3]:
# Import data
cases_raw = pd.read_csv(RAW_DATA_PATH + 'RAW_us_confirmed_cases.csv/RAW_us_confirmed_cases.csv')
mandates_raw = pd.read_csv('https://data.cdc.gov/resource/62d6-pm5i.csv')
masks_raw = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')
print(cases_raw.head())
print(mandates_raw.head())
print(masks_raw.head())

  Province_State    Admin2       UID iso2 iso3  code3    FIPS Country_Region  \
0        Alabama   Autauga  84001001   US  USA    840  1001.0             US   
1        Alabama   Baldwin  84001003   US  USA    840  1003.0             US   
2        Alabama   Barbour  84001005   US  USA    840  1005.0             US   
3        Alabama      Bibb  84001007   US  USA    840  1007.0             US   
4        Alabama    Blount  84001009   US  USA    840  1009.0             US   
5        Alabama   Bullock  84001011   US  USA    840  1011.0             US   
6        Alabama    Butler  84001013   US  USA    840  1013.0             US   
7        Alabama   Calhoun  84001015   US  USA    840  1015.0             US   
8        Alabama  Chambers  84001017   US  USA    840  1017.0             US   
9        Alabama  Cherokee  84001019   US  USA    840  1019.0             US   

         Lat      Long_           Combined_Key  1/22/20  1/23/20  1/24/20  \
0  32.539527 -86.644082   Autauga, Alabama

- FIPS is the key for each unique county. It is stored as 1 number in the `cases_raw` dataset but two values in the `mandate_raw` data set. When `FIPS_State` is single-digit then 0 is appended to the end before concatenating with `FIPS_County`. If `FIPS_County` is single digit as well, then 0 is appended to the beginning. Ex: Alabama Autauga County has state FIP 1 and county FIP 1, so it becomes 1001 for the combined FIP.
- For time-series analysis each date should be in one column, with the value in another. Long vs. wide data.  

In [6]:
# identify Orange County, CA FIPs code
fip = cases_raw.loc[(cases_raw['Province_State'] == 'California') & (cases_raw['Admin2'] == 'Orange'), 'FIPS'].values[0]
# print(fip)

# Create new combined FIPS column for mandates data set.
# If state code is single digit, then adds trailing 0
mandates = mandates_raw.copy()
mandates['FIPS'] = mandates['fips_county'] + mandates['fips_state'].apply(lambda x: x * 100 if x > 9 else x * 1000)
# Convert mandates_raw column into datetime
mandates['date'] = pd.to_datetime(mandates['date'], format = '%m/%d/%Y')

In [11]:
# Pivot case data to long format
cases = pd.melt(cases_raw, id_vars = cases_raw.columns[0:11], var_name = 'date', value_name = 'cases')
cases['date'] = pd.to_datetime(cases['date'], format = '%m/%d/%y')

In [14]:
print(cases.columns)
print(mandates.columns)
print(masks_raw.columns)

Index(['Province_State', 'Admin2', 'UID', 'iso2', 'iso3', 'code3', 'FIPS',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'date', 'cases'],
      dtype='object')
Index(['state_tribe_territory', 'county_name', 'fips_state', 'fips_county',
       'date', 'order_code', 'face_masks_required_in_public',
       'source_of_action', 'url', 'citation', 'FIPS'],
      dtype='object')
Index(['COUNTYFP', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS'], dtype='object')


In [24]:
# Create combined data set and write to data_clean assuming we are in A4 folder
covid_df = cases.merge(mandates, how = 'left', on = ['FIPS', 'date'])\
    .merge(masks_raw, how = 'left', left_on = 'FIPS', right_on = 'COUNTYFP')\
    .drop(['iso2', 'iso3', 'code3', 'Country_Region', 'COUNTYFP'], axis = 1)
covid_df.to_csv(CLEAN_DATA_PATH + 'us_covid-cases_mask-adoption_mandates.csv', index = False)

In [26]:
orange_df = covid_df.loc[covid_df['FIPS'] == fip, ]
orange_df.to_csv(CLEAN_DATA_PATH + 'orange-county_CA_covid-cases_mask-adoption_mandates.csv', index = False)

,Province_State,Admin2,UID,FIPS,Lat,Long_,Combined_Key,date,cases,state_tribe_territory,county_name,fips_state,fips_county,order_code,face_masks_required_in_public,source_of_action,url,citation,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
226,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2020-01-22,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
3568,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2020-01-23,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
6910,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2020-01-24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
10252,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2020-01-25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
13594,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2020-01-26,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152474,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2021-10-27,325607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
2155816,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2021-10-28,325921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
2159158,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2021-10-29,326246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
2162500,California,Orange,84006059,6059.0,33.701475,-117.7646,"Orange, California, US",2021-10-30,326246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023,0.021,0.046,0.156,0.754
